In [ ]:
%%capture
try:
    import google.colab  # noqa: F401
except ImportError:
    import ufl
    import dolfin
else:
    try:
        import ufl
        import dolfin
    except ImportError:
        !wget "https://fem-on-colab.github.io/releases/fenics-install.sh" -O "/tmp/fenics-install.sh" && bash "/tmp/fenics-install.sh"
        import ufl
        import dolfin

<h1>Gradient flows for coupling order parameters and mechanics</h1>

FENICS implementation of the examples from

Schmeller, L. & Peschka, D. (20223).

DOI: https://doi.org/10.1137/22M148478X
\\ 
<h3>General coupled gradient flow evolution</h3>

For a state variable $q=(u,\psi)$ with displacement $u:\Omega\to\mathbb{R}^d$ and phase field $\psi:\Omega\to\mathbb{R}^N$ we consider the following free energy 
\begin{align}
    \mathscr{F}(q) 
    = & 
    \int_{{\Omega}}W_{\rm elast}(F_e,\psi){\rm d}X
    + \int_{{\Omega}}W_{\rm phase}(\psi,\nabla\psi,F)
    {\rm d}X
\end{align}

for deformation gradient $F=\mathbb{I}_d+\nabla u=F_e F_p$ and with given plastic strain $F_p=F_p(\psi)$ and $F_e=FF_p^{-1}$.

We consider a Neo-Hookean elastic energy density 
\begin{align}
   W_{\rm elast}(F_e,\psi) = \frac{G}{2}\left(\text{tr}(F_e^\top F_e - \mathbb{I}_d) -2\log(\det(F_e))\right)+
 \frac{K}{2}\big(\det(F_e)-H\big)^2  
\end{align}
with bulk modulus $G=G(\psi)$ and phase volume $H=H(\psi)$ and inverse compressibility $K\in\mathbb{R}$. The remaining part of the free energy is 
\begin{align}
    W_\text{phase}(\psi,\nabla\psi,F) &= \left[\frac{1}{2}\nabla\psi\cdot \sigma\nabla\psi + W_\text{entropy}(\psi,F)\right]\det(F)
\end{align}
with second-order tensor $\sigma=\sigma(\psi,F)\in\mathbb{R}^{d\times d}$. 
Together with a given dissipation potential $R(q,\dot{q})$, different parameters, double-well or Flory-Huggins-type entropy $W_\text{entropy}$ and possible constraints $C(q,\nabla q)=0\in\mathbb{R}^M$ added through the Lagrangian in terms of $q_\lambda=(q,\lambda)$ with the optional Lagrange multiplier $\lambda:\Omega\to\mathbb{R}^M$
\begin{align}
  \mathscr{L}(q_\lambda)=\mathscr{F}(q)+\int_\Omega C(q,\nabla q)\cdot\lambda\,{\rm d}X
\end{align}
we consider the formal gradient flow evolution $\partial_t q =-\nabla_R \mathscr{F}(q)$, which we solve by a saddle-point problem generated by the minimization problem
\begin{align}
  \min_{v_\lambda=(v,\hat{\lambda})} \Big[R(q,v)+\langle \mathrm{D}\mathscr{L}(q_\lambda),v_\lambda\rangle\Big]%\right)
\end{align}
solved by incremental minimization as described in more detail in the manuscript.

We consider the formal gradient flow evolution $\partial_t q =-\nabla_R \mathscr{F}(q)$. This we solve by a saddle-point problem generated by the \DDD constrained  minimization problem of $\mathscr{R}$
\begin{align}
  \min_{q, \text{ s.t. } C=0} \Big[R(q^{k-1},\tfrac{1}{\tau}(q-q^{k-1}))+\mathscr{F}(q)\Big]
\quad\Rightarrow\quad
{\rm D }{q_\lambda}\Big[R(q^{k-1},\tfrac{1}{\tau}(q-q^{k-1}))+\mathscr{L}(q_\lambda)\Big]=0
\end{align} 
solved by incremental minimization as described in more detail in the manuscript. 

In [ ]:
filepath = './ExampleIntro/'

# put in a separate file
from IPython.display import HTML, display
def progress(value, max=100):
    return HTML("""
        Iterations: {value} of {max}
        <progress
            value='{value}'
            max='{max}',
            style='width: 100%'
        >
            {value}
        </progress>
    """.format(value=value, max=max))

from fenics import *
from matplotlib import pyplot as plt
import numpy as np

# should not appear in public version
def output_mesh(mesh):
    File(filepath + "mesh.xml") << mesh

def output_solution(q,n,t=0):
    File(filepath + "u"+str(n)+".xml") << project(q.sub(0),FunctionSpace(mesh,Vu)) 
    File(filepath + "psi"+str(n)+".xml") << project(q.sub(1),FunctionSpace(mesh,Vpsi))
    File(filepath + "eta_psi"+str(n)+".xml") << project(q.sub(2),FunctionSpace(mesh,U))

# Introductory example:

For the simplest example we consider a fixed two-dimensional domain $\Omega=(0,1)^2$ with homogeneous Dirichlet boundary conditions $u=0$ on $\partial\Omega$. We use one species $N=1$ and other constitutive relations / paramters

\begin{align}
G&=10\\
K&=10\\
H&=1\\
\sigma&=\varepsilon F^{-1}F^{-\top} \qquad \varepsilon=0.04\\
F_p&=(1+\alpha\psi)\mathbb{I}_2 \qquad \alpha=0.5 \\
W_\text{entropy}&=\frac{1}{4\varepsilon}(1-\psi^2)^2\\
R(q,\nabla q)&=\text{Cahn-Hilliard-type }R_\psi\text{ with }\mu=1\\
C(q,\nabla q)&=\text{no constraints in this example}
\end{align}

In [ ]:
# model parameters
N_T   = 1000            # time steps
G     = Constant(10.)   # elastic modulus
K     = Constant(10.)   # compression modulus
mu    = Constant(1.)    # Cahn-Hilliard mobility
eps   = Constant(0.05)  # phase field energy parameter
alpha = Constant(0.50)  # inelastic (plastic) coupling

# mesh parameters of tensorial Nx x Nx x 2 triangular elements mesh
Nx    = 64 
mesh  = RectangleMesh(Point(0,0),Point(1,1),Nx,Nx)

# define the function spaces
Vu    = VectorElement("P", mesh.ufl_cell(), 1) # V_u   = displacements
Vpsi  = FiniteElement("P", mesh.ufl_cell(), 1) # V_psi = order-parameter(s)
U     = FiniteElement("P", mesh.ufl_cell(), 1) # U     = forces
VxU   = FunctionSpace(mesh, MixedElement([Vu,Vpsi,U])) # tensor space of V x U

# homogeneous Dirichlet boundary conditions 
bc    = DirichletBC(VxU.sub(0), Constant((0, 0)), 'on_boundary')

In [ ]:
# incremental minimization for coupled finite strain elasticity + phase field
def incremental_minimization(old_q, tau):
    
    q  = Function(VxU)   
    dq = TestFunction(VxU)

    # current solution
    u, psi, eta_psi = split(q)
    # old solution
    old_u,old_psi,old_eta_psi = split(old_q)
    # test functions
    du,dpsi,deta_psi  = TestFunctions(VxU)

    # define continuum mechanics / plasticity variables
    d       = psi.geometric_dimension()
    I       = Identity(d)
    F       = I + grad(u)       # deformation gradient
    iFp     = 1/(1+alpha*psi)   # inverse of plastic strain
    Fe      = F*iFp             # strain = elastic strain * plastic strain
    Je      = det(Fe)           # Jacobian elastic strain
    J       = det(F)            # Jacobian strain
    gradpsi = inv(F).T*grad(psi)
    
    # define free energy F_free 
    W_elastic = (G/2)*(tr(Fe.T*Fe - I) - 2*ln(Je)) + (K/2)*(Je-1)**2
    W_phase   = ((eps/2)*inner(gradpsi,gradpsi) + 1/(4*eps)*((1-psi**2)**2))*J
    F_free    = (W_elastic + W_phase)*dx

    # backward Euler time derivative
    dot_psi = (psi-old_psi)/tau

    # add energy and M_psi operator
    Res  = derivative(F_free, q, dq) - inner(eta_psi,dpsi)*dx 
    
    # add bilinear form a_psi and M_psi*
    Res += inner(dot_psi, deta_psi)*dx + inner(mu*grad(eta_psi), grad(deta_psi))*dx
    
    # solve nonlinear problem using old solution as initial guess
    q.assign(old_q)    
    parameters["form_compiler"]["cpp_optimize"] = True
    parameters["form_compiler"]["quadrature_degree"] = 5

    solve(Res == 0, q, bc)   
    E_free    = assemble(F_free)
    E_elastic = assemble(W_elastic*dx)
    E_phase   = assemble(W_phase*dx)
    dissi     = assemble(tau*inner(dot_psi,eta_psi)*dx)

    return q,E_free,E_elastic,E_phase,dissi #,its          

In [ ]:
old_q = interpolate(Expression(("0","0","0.5*sin(6*x[0])*cos(5*x[1])+0.5*cos(3*x[0])*sin(7*x[1])","0"), degree = 2),VxU) # 2*(float)rand()-1
out   = display(progress(0, N_T), display_id=True)

tau      = 1e-6
t        = tau
q,E_free,E_elastic,E_phase,dissi = incremental_minimization(old_q, tau) 
energies = [[0,0.0,E_free,E_elastic,E_phase,dissi]]
old_q.assign(q)
output_mesh(mesh)
output_solution(q,0,t)
for n in range(N_T) :
    if n<50:
      tau = 0.01 / N_T
    else:
      tau = 0.1 / N_T
    t += tau
    q,E_free,E_elastic,E_phase,dissi = incremental_minimization(old_q, tau) 
    energies.append([n,t,E_free,E_elastic,E_phase,dissi])
    output_solution(q,n+1,t)
    out.update(progress( n+1 , N_T )) 
    old_q.assign(q)

E = list(zip(*energies))
np.save(filepath + "energies",E)

In [ ]:
u, psi, eta_psi = split(q)

E = list(zip(*energies))
np.save(filepath + "energies",E)

plt.figure(figsize=(21,7))
plt.subplot(1,3,1)
plot(psi)
plot(u)
plt.title("$u$ and $\psi$")

ALE.move(mesh,q.sub(0))
plt.subplot(1,3,2)
plot(psi)
plot(mesh)
ALE.move(mesh,project(-q,VxU).sub(0))
plt.title("$\psi$ and mesh")


time = np.array(E[1])[1:]
Free = np.array(E[2])[1:]
diss = np.array(E[5])[1:]

plt.subplot(1,3,3)
intdiss = np.cumsum(diss)

N0=0
scal=0
N3=N0+4*scal

time3 = np.array(E[1][N3:])[N3:]
Free3 = np.array(E[2][N3:])[N3:]
diss3 = np.array(E[5][N3:])[N3:]
intdiss3 = np.cumsum(diss3)
 
plt.plot(E[1],E[2])
plt.plot(E[1],E[3])
plt.plot(E[1],E[4])
plt.plot(time3,Free3[0]+(intdiss3-intdiss3[0]),'--')

plt.grid(True)
plt.xlabel("time $t$")
plt.ylabel("energy")
plt.legend(['$\mathscr{F}$', '$W_\mathrm{elast}$','$W_\mathrm{phase}$','$\mathscr{F}(q^0)+2 \\tau \sum_k  R_k$'])